<a href="https://colab.research.google.com/github/TasneemAhmed/COVID-19-Data-Analysis/blob/main/Covid_19_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Importing Packages**

In [ ]:
pip install --upgrade plotly 

     |████████████████████████████████| 25.3 MB 1.7 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [49]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# **2. Load Covid-19 Data**

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .csv files in the root.

# Search query reference (the id of folder):
listed = drive.ListFile({'q': "'1OYjGz6aKyIxFwS18otsmOhSFehvF1Zi3' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title worldometer_data.csv, id 1JK7HW_Fkb03dBFwwuzWWK3fQ1ZSYEs-W
title country_wise_latest.csv, id 1MoIv3s-NSg-Ww_iI3v5XhaU3DEfIVme-
title day_wise.csv, id 1_qXa-DdpX0-SNYdwTwW5cRSP1fFNpB79
title covid_19_clean_complete.csv, id 1agM3uysV_iF-ILLrqS9zTFdULDzFSGe9
title usa_country_wise.csv, id 18iufnHxGlP2Q4POz4cIZ26TVJ3QKvUC6
title full_grouped.csv, id 1UomjKOfw45O26sPJF78wvOZxmw9KlzMJ


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
path = '/content/drive/MyDrive/Covid-19 Data'
def read_file(file_name):
  return pd.read_csv(path + "/" + file_name)

In [ ]:
world_data  = read_file('worldometer_data.csv')
world_data

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,NaN,162804.0,NaN,2576668.0,NaN,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,NaN,98644.0,NaN,2047660.0,NaN,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,NaN,41638.0,NaN,1377384.0,NaN,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,NaN,14606.0,NaN,676357.0,NaN,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,NaN,9604.0,NaN,387316.0,NaN,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Montserrat,North America,4.992000e+03,13,NaN,1.0,NaN,10.0,NaN,2.0,NaN,2604.0,200.0,61.0,12220.0,NaN
205,Caribbean Netherlands,North America,2.624700e+04,13,NaN,NaN,NaN,7.0,NaN,6.0,NaN,495.0,NaN,424.0,16154.0,NaN
206,Falkland Islands,South America,3.489000e+03,13,NaN,NaN,NaN,13.0,NaN,0.0,NaN,3726.0,NaN,1816.0,520493.0,NaN
207,Vatican City,Europe,8.010000e+02,12,NaN,NaN,NaN,12.0,NaN,0.0,NaN,14981.0,NaN,NaN,NaN,Europe


In [ ]:
world_data.dtypes

Country/Region       object
Continent            object
Population          float64
TotalCases            int64
NewCases            float64
TotalDeaths         float64
NewDeaths           float64
TotalRecovered      float64
NewRecovered        float64
ActiveCases         float64
Serious,Critical    float64
Tot Cases/1M pop    float64
Deaths/1M pop       float64
TotalTests          float64
Tests/1M pop        float64
WHO Region           object
dtype: object

# **3. Preprocessing Data**

In [ ]:
#check first if there any nulls
world_data.isnull().values.any()

True

In [ ]:
#show sum of missing values in each column
world_data.isnull().sum()

Country/Region        0
Continent             1
Population            1
TotalCases            0
NewCases            205
TotalDeaths          21
NewDeaths           206
TotalRecovered        4
NewRecovered        206
ActiveCases           4
Serious,Critical     87
Tot Cases/1M pop      1
Deaths/1M pop        22
TotalTests           18
Tests/1M pop         18
WHO Region           25
dtype: int64

In [ ]:
#replace Continent & WHO Region empty values with 'Unknown' other columns replace empty values with 0
for index, value in world_data.dtypes.items(): 
    if value == 'object':
        world_data[index] = world_data[index].fillna('Unknown') 
    else:
        world_data[index] = world_data[index].fillna(0)

world_data

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,3.311981e+08,5032179,0.0,162804.0,0.0,2576668.0,0.0,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas
1,Brazil,South America,2.127107e+08,2917562,0.0,98644.0,0.0,2047660.0,0.0,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas
2,India,Asia,1.381345e+09,2025409,0.0,41638.0,0.0,1377384.0,0.0,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia
3,Russia,Europe,1.459409e+08,871894,0.0,14606.0,0.0,676357.0,0.0,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe
4,South Africa,Africa,5.938157e+07,538184,0.0,9604.0,0.0,387316.0,0.0,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Montserrat,North America,4.992000e+03,13,0.0,1.0,0.0,10.0,0.0,2.0,0.0,2604.0,200.0,61.0,12220.0,Unknown
205,Caribbean Netherlands,North America,2.624700e+04,13,0.0,0.0,0.0,7.0,0.0,6.0,0.0,495.0,0.0,424.0,16154.0,Unknown
206,Falkland Islands,South America,3.489000e+03,13,0.0,0.0,0.0,13.0,0.0,0.0,0.0,3726.0,0.0,1816.0,520493.0,Unknown
207,Vatican City,Europe,8.010000e+02,12,0.0,0.0,0.0,12.0,0.0,0.0,0.0,14981.0,0.0,0.0,0.0,Europe


In [ ]:
#show sum of missing values in each column
world_data.isnull().sum()

Country/Region      0
Continent           0
Population          0
TotalCases          0
NewCases            0
TotalDeaths         0
NewDeaths           0
TotalRecovered      0
NewRecovered        0
ActiveCases         0
Serious,Critical    0
Tot Cases/1M pop    0
Deaths/1M pop       0
TotalTests          0
Tests/1M pop        0
WHO Region          0
dtype: int64

# **4. Analysis Data**

**4.1 which country has maximum Total Cases, Total Deaths, TotalRecovered, and Active Cases**

In [ ]:
world_data.columns

Index(['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases',
       'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered',
       'ActiveCases', 'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop',
       'TotalTests', 'Tests/1M pop', 'WHO Region'],
      dtype='object')

In [ ]:
#With px.treemap, each row of the DataFrame is represented as a sector of the treemap.
target_columns = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases']
for col in target_columns:
  fig = px.treemap(world_data, path=[px.Constant("World"), 'Continent', 'Country/Region'], values=col, title='Treemap representaion of {} with different countries'.format(col))
  fig.show()

**4.2 Analysis day_wise.csv**

In [ ]:
day_wise = read_file('day_wise.csv')
day_wise

,Date,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,Deaths / 100 Cases,Recovered / 100 Cases,Deaths / 100 Recovered,No. of countries
0,2020-01-22,555,17,28,510,0,0,0,3.06,5.05,60.71,6
1,2020-01-23,654,18,30,606,99,1,2,2.75,4.59,60.00,8
2,2020-01-24,941,26,36,879,287,8,6,2.76,3.83,72.22,9
3,2020-01-25,1434,42,39,1353,493,16,3,2.93,2.72,107.69,11
4,2020-01-26,2118,56,52,2010,684,14,13,2.64,2.46,107.69,13
...,...,...,...,...,...,...,...,...,...,...,...,...
183,2020-07-23,15510481,633506,8710969,6166006,282756,9966,169714,4.08,56.16,7.27,187
184,2020-07-24,15791645,639650,8939705,6212290,281164,6144,228736,4.05,56.61,7.16,187
185,2020-07-25,16047190,644517,9158743,6243930,255545,4867,219038,4.02,57.07,7.04,187
186,2020-07-26,16251796,648621,9293464,6309711,204606,4104,134721,3.99,57.18,6.98,187


In [ ]:
day_wise.columns

Index(['Date', 'Confirmed', 'Deaths', 'Recovered', 'Active', 'New cases',
       'New deaths', 'New recovered', 'Deaths / 100 Cases',
       'Recovered / 100 Cases', 'Deaths / 100 Recovered', 'No. of countries'],
      dtype='object')

In [ ]:
px.line(data_frame=day_wise, x='Date', y=['Confirmed', 'Deaths', 'Recovered', 'Active'], title='COVID-19 Cases', template='plotly_dark')

**4.3 Visualize the first 20 countries aganist Population/Total Tests ratio**

In [ ]:
world_data.columns

Index(['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases',
       'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered',
       'ActiveCases', 'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop',
       'TotalTests', 'Tests/1M pop', 'WHO Region'],
      dtype='object')

In [ ]:
world_data['TotalPopulation / TotalTests'] = world_data['Population']/world_data['TotalTests']
world_data.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region,TotalPopulation / TotalTests
0,USA,North America,3.311981e+08,5032179,0.0,162804.0,0.0,2576668.0,0.0,2292707.0,18296.0,15194.0,492.0,63139605.0,190640.0,Americas,5.245489
1,Brazil,South America,2.127107e+08,2917562,0.0,98644.0,0.0,2047660.0,0.0,771258.0,8318.0,13716.0,464.0,13206188.0,62085.0,Americas,16.106896
2,India,Asia,1.381345e+09,2025409,0.0,41638.0,0.0,1377384.0,0.0,606387.0,8944.0,1466.0,30.0,22149351.0,16035.0,South-EastAsia,62.365033
3,Russia,Europe,1.459409e+08,871894,0.0,14606.0,0.0,676357.0,0.0,180931.0,2300.0,5974.0,100.0,29716907.0,203623.0,Europe,4.911040
4,South Africa,Africa,5.938157e+07,538184,0.0,9604.0,0.0,387316.0,0.0,141264.0,539.0,9063.0,162.0,3149807.0,53044.0,Africa,18.852446


In [ ]:
#iloc:select rows from dataframe
px.bar(data_frame=world_data.iloc[0:20],x='Country/Region', y='TotalPopulation / TotalTests', title='Population to Tests done Ratio', color='Country/Region')

**4.4 Visualize the 20 first countries affected badly by corona**

In [ ]:
px.bar(data_frame=world_data.iloc[0:20], x='Country/Region', y='Serious,Critical', title='Serious,Critical numbers in some of Countries',color='Country/Region')

**4.5 Visualize Countries aganist TotalCases, NewCases,TotalDeaths, NewDeaths TotalRecovered, NewRecovered**

In [ ]:
px.bar(data_frame=world_data.iloc[0:20], x='Country/Region', y=['TotalCases','TotalDeaths', 'TotalRecovered'])

In [ ]:
world_data.columns

Index(['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases',
       'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered',
       'ActiveCases', 'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop',
       'TotalTests', 'Tests/1M pop', 'WHO Region',
       'TotalPopulation / TotalTests'],
      dtype='object')

**4.6 Create different visualization(Country/Region VS TotalCases, TotalRecovered, TotalDeath,and ActiveCases)**

In [ ]:
def create_viz(column_name):
  sorted_df_desc = world_data.sort_values(by=column_name, ascending=False)
  fig = px.bar(data_frame=sorted_df_desc.iloc[0:20], y='Country/Region', x=column_name, color=column_name, text=column_name)
  fig.update_layout(template='plotly_dark', title='{} Top in 20 Countries'.format(column_name))
  fig.show()

In [ ]:
create_viz('TotalCases')

In [ ]:
create_viz('TotalDeaths')

In [ ]:
create_viz('ActiveCases')

In [ ]:
create_viz('TotalRecovered')

**4.7 Show different cases for first 15 countries**

In [30]:
slice = world_data[0:15]
labels = slice['Country/Region']
target_cases = ['TotalCases', 'TotalDeaths', 'TotalRecovered', 'ActiveCases']

for case in target_cases:
  fig = px.pie(data_frame=slice, names=labels, values=case, title='{} for first 15 countries'.format(case))
  fig.show()

**4.8 Show different ratios for every country**

In [45]:
def ratio_viz(col1, col2):
  ratio_result = world_data[col1] / world_data[col2]
  fig = px.bar(data_frame=world_data, x=world_data['Country/Region'], y=ratio_result, title='{}/{} for every Country'.format(col1, col2), template='plotly_dark', labels={
                     "y": "{}/{} ratio".format(col1, col2),
                     "color": "ratio"
                 },color=ratio_result)
  fig.show()

In [40]:
world_data.columns

Index(['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases',
       'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered',
       'ActiveCases', 'Serious,Critical', 'Tot Cases/1M pop', 'Deaths/1M pop',
       'TotalTests', 'Tests/1M pop', 'WHO Region',
       'TotalPopulation / TotalTests'],
      dtype='object')

In [46]:
death_to_confirmed = ratio_viz('TotalDeaths', 'TotalCases')
death_to_confirmed

In [47]:
serious_to_death = ratio_viz('Serious,Critical', 'TotalDeaths')
serious_to_death

In [48]:
tests_to_recovered = ratio_viz('TotalTests', 'TotalRecovered')
tests_to_recovered

**4.9 Visualize different cases aganist date but for specific Country**

In [50]:
#make this analysis on full_grouped.csv
grouped_data  = read_file('full_grouped.csv')
grouped_data

,Date,Country/Region,Confirmed,Deaths,Recovered,Active,New cases,New deaths,New recovered,WHO Region
0,2020-01-22,Afghanistan,0,0,0,0,0,0,0,Eastern Mediterranean
1,2020-01-22,Albania,0,0,0,0,0,0,0,Europe
2,2020-01-22,Algeria,0,0,0,0,0,0,0,Africa
3,2020-01-22,Andorra,0,0,0,0,0,0,0,Europe
4,2020-01-22,Angola,0,0,0,0,0,0,0,Africa
...,...,...,...,...,...,...,...,...,...,...
35151,2020-07-27,West Bank and Gaza,10621,78,3752,6791,152,2,0,Eastern Mediterranean
35152,2020-07-27,Western Sahara,10,1,8,1,0,0,0,Africa
35153,2020-07-27,Yemen,1691,483,833,375,10,4,36,Eastern Mediterranean
35154,2020-07-27,Zambia,4552,140,2815,1597,71,1,465,Africa


In [54]:
def country_viz(country, case):
  data = grouped_data[grouped_data['Country/Region'] == country]
  data = data.loc[:, ['Date', case]]

  fig = px.scatter(data, x="Date", y=case, color=case, size=case, title='{} Cases in {} Over Year'.format(case, country), template= 'plotly_dark')
  fig.show()

In [52]:
grouped_data.columns

Index(['Date', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered', 'Active',
       'New cases', 'New deaths', 'New recovered', 'WHO Region'],
      dtype='object')

In [55]:
country_viz('Zambia', 'Confirmed')

In [56]:
country_viz('Egypt', 'Deaths')

In [57]:
country_viz('Zimbabwe', 'New deaths')